# Goal:

- provide a plaintext interface to analyze step-by-step what is happening in the encrypted code

- Used as a Python sanity check because I'm not that familiar with R.

In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, confusion_matrix
from numba import njit

np.seterr(all='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Load and Process the Data

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def load_data(num_samples, compare_to_r_ref):
    x_file = "../train_data/X_norm_1024.csv"
    y_file = "../train_data/y_1024.csv"
    train_x = pd.read_csv(x_file)
    train_x = train_x.to_numpy()[:num_samples]
    train_y = pd.read_csv(y_file)
    train_y = train_y.to_numpy()[:num_samples]
    print(f"{bcolors.OKGREEN}Using subsampled data to compare Python-C++{bcolors.ENDC}")
    print(f"{bcolors.OKGREEN}Reading in {x_file}, {y_file} {bcolors.ENDC}")

    print(f"Train X shape is: {train_x.shape}")
    print(f"Train y shape is: {train_y.shape}")
    return train_x, train_y

In [3]:
NUM_SAMPLES = -1
COMPARE_TO_R_REF = False
lr = 0.1
mu = 0.1
train_x, train_y = load_data(
    num_samples=NUM_SAMPLES,
    compare_to_r_ref=COMPARE_TO_R_REF
)

# Same shape as Marcelo's reference code
betas = np.zeros((10, ))
betas = np.random.uniform(-0.1, 0.1, 10)
betas = np.random.randn(10) * 0.01

Using subsampled data to compare Python-C++
Reading in ../train_data/X_norm_1024.csv, ../train_data/y_1024.csv 
Train X shape is: (1023, 10)
Train y shape is: (1023, 1)


In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def fwd(train_x, betas, dbg=False):
    preds = train_x @ betas   # A vector of linear_predictions/logits z = train_x @ weights
    if dbg:
        print(f"Logits: {preds}")
    return np.expand_dims(sigmoid(preds), -1)   # Shape: (m, 1)

def calculate_gradient(train_x, train_y, betas, fwd, dbg):
    preds = fwd(train_x, betas, dbg)   # A vector of logistic predictions y_hat = sigmoid(z)
    gradient = -train_x.T @ (train_y - preds) / len(train_y)
    return gradient   # Shape: (10, 1) == Rows correspond to num_features
    ## This function is used to update the values of betas: w_new = w_old + lr * gradient

def cost(x, y, theta):
    m = x.shape[0]
    h = sigmoid(np.matmul(x, theta))   # h: hypothesis, basically preds/y_hat
    t1 = np.matmul(-y.T, np.log(h))
    t2_a = (1 - y.T)
    t2_b = np.log(np.clip(1 - h, 0.000000000000001, np.max(1 - h)))  # Used to get numerical issues
    ## np.clip() function prevents computing the log of 0, by taking the minimum of 1e-15.
    t2 = np.matmul(t2_a, t2_b)

    return ((t1 - t2) / m)[0]   # Shape: (1,) == scalar value

def nesterov(betas, epochs, lr, mu, train_x, train_y):
    import copy

    phi = copy.deepcopy(betas)
    theta = copy.deepcopy(betas)

    nesterov_loss = [0 for _ in range(epochs)]
    # for i in tqdm.trange(epochs):
    for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)

        ## Assign updated weights into phi_prime
        phi_prime = theta - lr * np.squeeze(gradient)   # np.squeeze() removes single dimensions --> shape (10,)
        
        ## Nesterov acceleration process
        if i == 0:
            theta = phi_prime
        else:
            ## If current updated weight (phi_prime) < previous weight (phi), 
            ## The updated weight theta will be even smaller.
            theta = phi_prime + mu * (phi_prime - phi)
        phi = phi_prime   # phi is then the weight of the previous epoch/update
        loss = cost(train_x, train_y, theta)
        nesterov_loss[i] = loss

        # print(f"New loss: {cost(train_x, train_y, v)[0]}")
    return nesterov_loss, theta, phi


In [5]:
losses, theta, phi = nesterov(betas, 200, lr, mu, train_x, train_y)

In [6]:
losses

## zero initialized betas    : cost = 0.4087381183990924
## uniform initialized betas : cost = 0.40867096414830645
## randn initialized betas   : cost = 0.40850863604124726

[0.6812155165482665,
 0.6701553917502939,
 0.6597774766250945,
 0.6500832775052069,
 0.6410014956245764,
 0.6324669733017375,
 0.6244252707678345,
 0.6168308106865237,
 0.6096449646013661,
 0.6028345610285551,
 0.5963707562049805,
 0.5902281809178755,
 0.5843842937150668,
 0.5788188876879484,
 0.5735137113665079,
 0.5684521744304052,
 0.5636191165796894,
 0.5590006236029212,
 0.55458387889821,
 0.5503570418116154,
 0.5463091464403759,
 0.5424300162201237,
 0.5387101908368336,
 0.5351408628958031,
 0.531713822430348,
 0.5284214078075234,
 0.5252564619350135,
 0.522212292927371,
 0.5192826385765259,
 0.5164616341094356,
 0.5137437828183132,
 0.5111239292257703,
 0.5085972345054297,
 0.5061591539231828,
 0.5038054160989116,
 0.5015320039158202,
 0.49933513692640485,
 0.497211255121923,
 0.49515700394697815,
 0.49316922045325373,
 0.49124492049702684,
 0.4893812868942434,
 0.4875756584549551,
 0.4858255198259803,
 0.4841284920769497,
 0.4824823239705289,
 0.48088488386268663,
 0.4793341521

In [7]:
theta

array([-1.25031844,  0.2002066 , -1.18935318,  0.42938639, -0.64430861,
       -0.05019797, -0.08696701, -0.36465605,  0.11685985,  0.4454505 ])

In [8]:
phi

array([-1.25004521,  0.20012118, -1.18916174,  0.42939299, -0.64428704,
       -0.05027591, -0.08692788, -0.36454147,  0.11683475,  0.44529883])

In [9]:
pred = fwd(train_x, theta, dbg=False)

## Decision (Threshold = 0.5)
train_y_hat = (pred >= 0.5).astype(int)

In [10]:
train_y_hat.shape

(1023, 1)

In [11]:
train_y.shape

(1023, 1)

In [12]:
confusion_matrix(train_y, train_y_hat)

array([[463,  48],
       [136, 376]], dtype=int64)

In [13]:
accuracy_score(train_y, train_y_hat)

0.820136852394917